## This Notebook is to showcase the deploy function of ModelBuilder. 

As ModelBuilder becomes the main interface for all inference related changes, this deploy function is one step further to provide an enhanced experience over Model.deploy(). 
The deploy function mainly provides 2 advantages:
1. Simplification of parameters - Reducing the total parameters required from 22 to 7. 
2. Validation/Error Handling with enum to make sure the deployment mode related parameters are provided in the call

### Basic Setup

In [1]:
from sagemaker import Session, get_execution_role

sagemaker_session = Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name
bucket = sagemaker_session.default_bucket()


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/nargokul/Library/Application Support/sagemaker/config.yaml


### Create Training Job and Model Builder

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import pandas as pd

# Get IRIS Data

iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

import os

# Prepare Data

os.makedirs('./data', exist_ok=True)

iris_df = iris_df[['target'] + [col for col in iris_df.columns if col != 'target']]

train_data, test_data = train_test_split(iris_df, test_size=0.2, random_state=42)

train_data.to_csv('./data/train.csv', index=False, header=False)
test_data.to_csv('./data/test.csv', index=False, header=False)

# Remove the target column from the testing data. We will use this to call invoke_endpoint later
test_data_no_target = test_data.drop('target', axis=1)

prefix = "DEMO-scikit-iris"
TRAIN_DATA = "train.csv"
DATA_DIRECTORY = "data"

train_input = sagemaker_session.upload_data(
    DATA_DIRECTORY, bucket=bucket, key_prefix="{}/{}".format(prefix, DATA_DIRECTORY)
)

s3_input_path = "s3://{}/{}/data/{}".format(bucket, prefix, TRAIN_DATA)
s3_output_path = "s3://{}/{}/output".format(bucket, prefix)

print(s3_input_path)
print(s3_output_path)

s3://sagemaker-us-west-2-211125564141/DEMO-scikit-iris/data/train.csv
s3://sagemaker-us-west-2-211125564141/DEMO-scikit-iris/output


In [3]:
from sagemaker_core.main.shapes import AlgorithmSpecification, Channel, DataSource, S3DataSource,OutputDataConfig, ResourceConfig, StoppingCondition
import uuid
from sagemaker.serve.builder.model_builder import  ModelBuilder, BatchTransformInferenceConfig
import pandas as pd
import numpy as np
from sagemaker.serve import InferenceSpec, SchemaBuilder
from sagemaker_core.main.resources import TrainingJob
from xgboost import XGBClassifier

image = "433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest"

class XGBoostSpec(InferenceSpec):
    def load(self, model_dir: str):
        print(model_dir)
        model = XGBClassifier()
        model.load_model(model_dir + "/xgboost-model")
        return model

    def invoke(self, input_object: object, model: object):
        prediction_probabilities = model.predict_proba(input_object)
        predictions = np.argmax(prediction_probabilities, axis=1)
        return predictions
    
data = {
    'Name': ['Alice', 'Bob', 'Charlie']
}
df = pd.DataFrame(data)
training_job_name  = str(uuid.uuid4())
schema_builder = SchemaBuilder(sample_input=df, sample_output=df)

training_job = TrainingJob.create(
    training_job_name=training_job_name,
    hyper_parameters={
        'objective': 'multi:softmax',
        'num_class': '3',
        'num_round': '10',
        'eval_metric': 'merror'
    },
    algorithm_specification=AlgorithmSpecification(
        training_image=image,
        training_input_mode='File'
    ),
    role_arn=role,
    input_data_config=[
        Channel(
            channel_name='train',
            content_type='csv',
            compression_type='None',
            record_wrapper_type='None',
            data_source=DataSource(
                s3_data_source=S3DataSource(
                    s3_data_type='S3Prefix',
                    s3_uri=s3_input_path,
                    s3_data_distribution_type='FullyReplicated'
                )
            )
        )
    ],
    output_data_config=OutputDataConfig(
        s3_output_path=s3_output_path
    ),
    resource_config=ResourceConfig(
        instance_type='ml.m4.xlarge',
        instance_count=1,
        volume_size_in_gb=30
    ),
    stopping_condition=StoppingCondition(
        max_runtime_in_seconds=600
    )
)
training_job.wait()

model_builder = ModelBuilder(
    model_path=training_job.model_artifacts.s3_model_artifacts,
    role_arn=role,
    inference_spec=XGBoostSpec(),
    image_uri=image,
    schema_builder=schema_builder,
    instance_type="ml.c6i.xlarge"
)
model_builder.build()


/Library/Python/3.9/site-packages/pydantic/_internal/_fields.py:200: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[10/30/24 13:10:11] INFO     Creating training_job resource.                                     ]8;id=847076;file:///Library/Python/3.9/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=443564;file:///Library/Python/3.9/site-packages/sagemaker_core/main/resources.py#26309\26309]8;;\

                    WARNING  No boto3 session provided. Creating a new session.                        ]8;id=990982;file:///Library/Python/3.9/site-packages/sagemaker_core/main/utils.py\utils.py]8;;\:]8;id=537669;file:///Library/Python/3.9/site-packages/sagemaker_core/main/utils.py#336\336]8;;\

                    WARNING  No region provided. Using default region.                                 ]8;id=825815;file:///Library/Python/3.9/site-packages/sagemaker_core/main/utils.py\utils.py]8;;\:]8;id=145527;file:///Library/Python/3.9/site-packages/sagemaker_core/main/utils.py#340\340]8;;\

                    WARNING  No config provided. Using default config.                                 ]8;id=288203;file:///Library/Python/3.9/site-packages/sagemaker_core/main/utils.py\utils.py]8;;\:]8;id=337502;file:///Library/Python/3.9/site-packages/sagemaker_core/main/utils.py#344\344]8;;\

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=439359;file:///Library/Python/3.9/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=831253;file:///Library/Python/3.9/site-packages/botocore/credentials.py#1278\1278]8;;\

Output()

[10/30/24 13:12:53] INFO     Final Resource Status: Completed                                    ]8;id=750619;file:///Library/Python/3.9/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=635418;file:///Library/Python/3.9/site-packages/sagemaker_core/main/resources.py#26595\26595]8;;\

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=493314;file:///Library/Python/3.9/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=633308;file:///Library/Python/3.9/site-packages/botocore/credentials.py#1278\1278]8;;\

ModelBuilder: INFO:     Either inference spec or model is provided. ModelBuilder is not handling MLflow model input
ModelBuilder: INFO:     Skipping auto detection as the image uri is provided 433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest


#### Real Time Deployment

In [4]:
real_time_predictor = model_builder.deploy(
    initial_instance_count=1)

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.


                    INFO     Creating model with name: model-name-f76fb1fa96fa11efb5372ad633a99cdc  ]8;id=53030;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=517463;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#4025\4025]8;;\

[10/30/24 13:12:54] INFO     Creating endpoint-config with name                                     ]8;id=492611;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=991435;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#5817\5817]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-12-54-372                       

                    INFO     Creating endpoint with name                                            ]8;id=428897;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=73333;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#4639\4639]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-12-54-372                       

[10/30/24 13:15:26] INFO     Arguments: serve                                                       ]8;id=746109;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=339045;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8308\8308]8;;\

...

                    INFO     [2024-10-30 20:15:17 +0000] [24] [INFO] Booting worker with pid: 24    ]8;id=6848;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=288255;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8308\8308]8;;\

                    INFO     /opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65 ]8;id=941362;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=289797;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8308\8308]8;;\
                             : MonkeyPatchWarning: Monkey-patching ssl after ssl has already been                  
                             imported may lead to errors, including RecursionError on Python 3.6.                  
                             It may also silently lead to incorrect behaviour on Python 3.7. Please                
                             monkey-patch earlier. See                                                             
                             https://github.com/gevent/gevent/issues/1016. Modules that had direct                 
                             imports (NOT patched): ['urllib3.util                                                 
                             (/opt/amazon/lib/python3.7/site-packages/urllib3/util/__init__.py)',                  
                             'urllib3.util.ssl_                                                                    
                             (/opt/amazon/lib/python3.7/site-packages/urllib3/util/ssl_.py)'].                     
                               monkey.patch_all(subprocess=True)                                                   

                    INFO     [2024-10-30:20:15:17:INFO] Model loaded successfully for worker : 24   ]8;id=70291;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=909473;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8308\8308]8;;\

[10/30/24 13:15:59] INFO     Created endpoint with name                                             ]8;id=778192;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=467032;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8294\8294]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-12-54-372                       

ModelBuilder: DEBUG:     ModelBuilder metrics emitted.


#### Serverless Deployment

In [5]:
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

serverless_predictor = model_builder.deploy(
    inference_config=ServerlessInferenceConfig())

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.


[10/30/24 13:16:00] INFO     Creating model with name: model-name-f76fb1fa96fa11efb5372ad633a99cdc  ]8;id=245422;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=544458;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#4025\4025]8;;\

[10/30/24 13:16:01] WARNING  Using already existing model:                                          ]8;id=30512;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=969586;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#4036\4036]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a99cdc                                           

                    INFO     Creating endpoint-config with name                                     ]8;id=503479;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=809762;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#5817\5817]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-16-01-492                       

[10/30/24 13:16:02] INFO     Creating endpoint with name                                            ]8;id=858959;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=350075;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#4639\4639]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-16-01-492                       

...

                    INFO       monkey.patch_all(subprocess=True)                                    ]8;id=382247;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=161965;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8308\8308]8;;\

                    INFO     [2024-10-30:20:17:57:INFO] Model loaded successfully for worker : 31   ]8;id=415016;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=24361;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8308\8308]8;;\

                    INFO     [2024-10-30:20:17:57:INFO] Model loaded successfully for worker : 30   ]8;id=927407;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=387335;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8308\8308]8;;\

[10/30/24 13:18:14] INFO     Created endpoint with name                                             ]8;id=549964;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=34785;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8294\8294]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-16-01-492                       

ModelBuilder: DEBUG:     ModelBuilder metrics emitted.


#### Async Deployment

In [6]:
from sagemaker.s3_utils import s3_path_join
from sagemaker.async_inference import AsyncInferenceConfig

async_predictor = model_builder.deploy(
    inference_config=AsyncInferenceConfig(
        output_path=s3_path_join("s3://", bucket, "async_inference/output")),
)

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.


[10/30/24 13:18:15] INFO     Creating model with name: model-name-f76fb1fa96fa11efb5372ad633a99cdc  ]8;id=96650;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=60711;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#4025\4025]8;;\

[10/30/24 13:18:16] WARNING  Using already existing model:                                          ]8;id=492060;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=45096;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#4036\4036]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a99cdc                                           

[10/30/24 13:18:19] INFO     Creating endpoint-config with name                                     ]8;id=21634;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=59349;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#5817\5817]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-18-16-602                       

[10/30/24 13:18:20] INFO     Creating endpoint with name                                            ]8;id=747862;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=622744;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#4639\4639]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-18-16-602                       

...

                    INFO     [2024-10-30 20:20:51 +0000] [24] [INFO] Booting worker with pid: 24    ]8;id=564172;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=217009;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8308\8308]8;;\

                    INFO     /opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65 ]8;id=669354;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=873117;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8308\8308]8;;\
                             : MonkeyPatchWarning: Monkey-patching ssl after ssl has already been                  
                             imported may lead to errors, including RecursionError on Python 3.6.                  
                             It may also silently lead to incorrect behaviour on Python 3.7. Please                
                             monkey-patch earlier. See                                                             
                             https://github.com/gevent/gevent/issues/1016. Modules that had direct                 
                             imports (NOT patched): ['urllib3.util                                                 
                             (/opt/amazon/lib/python3.7/site-packages/urllib3/util/__init__.py)',                  
                             'urllib3.util.ssl_                                                                    
                             (/opt/amazon/lib/python3.7/site-packages/urllib3/util/ssl_.py)'].                     
                               monkey.patch_all(subprocess=True)                                                   

                    INFO     [2024-10-30:20:20:51:INFO] Model loaded successfully for worker : 24   ]8;id=901867;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=827841;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8308\8308]8;;\

[10/30/24 13:21:26] INFO     Created endpoint with name                                             ]8;id=56717;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=391476;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#8294\8294]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-18-16-602                       

ModelBuilder: DEBUG:     ModelBuilder metrics emitted.


#### Batch Deployment

In [ ]:
from sagemaker.s3_utils import s3_path_join


batch_predictor = model_builder.deploy(
    initial_instance_count=1,
    inference_config=BatchTransformInferenceConfig(
        instance_count=1,
        instance_type='ml.m5.large',
        output_path=s3_path_join("s3://", bucket, "async_inference/output"),
    )
)

#### MultiModel Deployment

Similar syntax applies for Multi-Container Endpoint as well

In [ ]:
from sagemaker.compute_resource_requirements import ResourceRequirements

multi_model_predictor = model_builder.deploy(
    inference_config = ResourceRequirements(
        requests={
            "num_cpus": 0.5,
            "memory": 512,
            "copies": 2,
        },
        limits={},
    ),
    initial_instance_count=1)

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.


[10/30/24 13:30:58] INFO     Creating endpoint-config with name                                     ]8;id=987242;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=233313;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#5817\5817]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-30-57-438                       

                    INFO     Creating endpoint with name                                            ]8;id=10549;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py\session.py]8;;\:]8;id=757746;file:///Users/nargokul/workspace/sagemaker-python-sdk-staging/sagemaker-python-sdk-staging/src/sagemaker/session.py#4639\4639]8;;\
                             model-name-f76fb1fa96fa11efb5372ad633a9-2024-10-30-20-30-57-438                       